In [6]:
import pandas as pd
import numpy as np
import pymysql
import getpass

# Pandas - SQL, Joins, Groupbys

1. connect to mysql database
2. rewrite sql queries into pandas
3. possibly - column transformations, decoding

## 1. Connect to MySQL database

In [7]:
pw = getpass.getpass()

········


In [8]:
conn = pymysql.connect(host="localhost",
                       port=3306,
                       user="root",
                       passwd=pw,
                       db="olist")

In [9]:
conn

In [11]:
pd.read_sql("SELECT * FROM orders LIMIT 10", conn).dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

## 2. Rewrite SQL queries into pandas
For each product catagory, get the number of items sold, total revenue (sum of price), and the revenue per item sold

    (I) Joining `order_items` and `products`

![title](query_1.png)

In [12]:
order_items = pd.read_sql("SELECT * FROM order_items", conn)
products = pd.read_sql("SELECT * FROM products", conn)

In [13]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 11:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 13:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 15:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 12:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 14:57:51,199.90,18.14


In [14]:
products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [15]:
order_items.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

In [16]:
products.dtypes

product_id                     object
product_category_name          object
product_name_length           float64
product_description_length    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [20]:
#subsetting with the join will only save that subset to the variable
order_products = order_items.merge(right=products,
                                   how="left",
                                   on="product_id")[["order_id",
                                                     "order_item_id",
                                                     "product_id",
                                                     "price",
                                                     "product_category_name"]] #subsetting

In [21]:
order_products.head()

,order_id,order_item_id,product_id,price,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,moveis_decoracao
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,perfumaria
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,ferramentas_jardim


    (II) Adding a GROUPBY

![title](group_by.png)

In [22]:
order_products.groupby("product_category_name")

In [23]:
per_category = (order_products
                .groupby("product_category_name")
                .agg({"price":"sum",
                      "order_id":"count"}))

In [24]:
per_category.head()

,price,order_id
product_category_name,,
agro_industria_e_comercio,72530.47,212
alimentos,29393.41,510
alimentos_bebidas,15179.48,278
artes,24202.64,209
artes_e_artesanato,1814.01,24


In [27]:
order_products.groupby("product_category_name").sum()[["price"]]

,price
product_category_name,
agro_industria_e_comercio,72530.47
alimentos,29393.41
alimentos_bebidas,15179.48
artes,24202.64
artes_e_artesanato,1814.01
...,...
sinalizacao_e_seguranca,21509.23
tablets_impressao_imagem,7528.41
telefonia,323667.53


In [29]:
order_products.groupby("product_category_name").sum()

,order_item_id,price
product_category_name,,
agro_industria_e_comercio,265,72530.47
alimentos,609,29393.41
alimentos_bebidas,365,15179.48
artes,230,24202.64
artes_e_artesanato,26,1814.01
...,...,...
sinalizacao_e_seguranca,307,21509.23
tablets_impressao_imagem,89,7528.41
telefonia,5161,323667.53


In [28]:
order_products.groupby("product_category_name").count()

,order_id,order_item_id,product_id,price
product_category_name,,,,
agro_industria_e_comercio,212,212,212,212
alimentos,510,510,510,510
alimentos_bebidas,278,278,278,278
artes,209,209,209,209
artes_e_artesanato,24,24,24,24
...,...,...,...,...
sinalizacao_e_seguranca,199,199,199,199
tablets_impressao_imagem,83,83,83,83
telefonia,4545,4545,4545,4545


In [30]:
(order_products
 .groupby("product_category_name")
 .agg({"price": np.sum,
 "order_id":np.size}))

,price,order_id
product_category_name,,
agro_industria_e_comercio,72530.47,212
alimentos,29393.41,510
alimentos_bebidas,15179.48,278
artes,24202.64,209
artes_e_artesanato,1814.01,24
...,...,...
sinalizacao_e_seguranca,21509.23,199
tablets_impressao_imagem,7528.41,83
telefonia,323667.53,4545


In [46]:
#can you pass a list of functions to an agg - and it works! but need to flatten column names
(order_products
 .groupby("product_category_name")
 .agg({"price": ["sum", "count"],
       "order_id":"count"}))

,price,order_id
product_category_name,,
agro_industria_e_comercio,212,212
alimentos,510,510
alimentos_bebidas,278,278
artes,209,209
artes_e_artesanato,24,24
...,...,...
sinalizacao_e_seguranca,199,199
tablets_impressao_imagem,83,83
telefonia,4545,4545


In [33]:
per_category

,price,order_id
product_category_name,,
agro_industria_e_comercio,72530.47,212
alimentos,29393.41,510
alimentos_bebidas,15179.48,278
artes,24202.64,209
artes_e_artesanato,1814.01,24
...,...,...
sinalizacao_e_seguranca,21509.23,199
tablets_impressao_imagem,7528.41,83
telefonia,323667.53,4545


In [34]:
per_category.loc["artes", ]

price       24202.64
order_id      209.00
Name: artes, dtype: float64

In [35]:
per_category.columns

Index(['price', 'order_id'], dtype='object')

In [36]:
per_category.loc["artes", "price"]

24202.639999999992

In [37]:
per_category.reset_index()

,product_category_name,price,order_id
0,agro_industria_e_comercio,72530.47,212
1,alimentos,29393.41,510
2,alimentos_bebidas,15179.48,278
3,artes,24202.64,209
4,artes_e_artesanato,1814.01,24
...,...,...,...
68,sinalizacao_e_seguranca,21509.23,199
69,tablets_impressao_imagem,7528.41,83
70,telefonia,323667.53,4545
71,telefonia_fixa,59583.00,264


Combine all the steps

In [41]:
(order_items
 .merge(products, how="left", on="product_id")
 .groupby("product_category_name")
 .agg({"price":"sum",
       "order_id":"count"})
#       "product_id":""}) would it work without explicitly saying which agg function? no
 .reset_index())

,product_category_name,price,order_id
0,agro_industria_e_comercio,72530.47,212
1,alimentos,29393.41,510
2,alimentos_bebidas,15179.48,278
3,artes,24202.64,209
4,artes_e_artesanato,1814.01,24
...,...,...,...
68,sinalizacao_e_seguranca,21509.23,199
69,tablets_impressao_imagem,7528.41,83
70,telefonia,323667.53,4545
71,telefonia_fixa,59583.00,264


In [43]:
random_name = (order_items
               .merge(products, how="left", on="product_id")
               .groupby("product_category_name")
               .agg({"price":"sum",
                     "order_id":"count"})
               .reset_index()
               .rename(columns={"price":"revenue",
                  "order_id":"items_sold"}))

In [44]:
random_name.head()

,product_category_name,revenue,items_sold
0,agro_industria_e_comercio,72530.47,212
1,alimentos,29393.41,510
2,alimentos_bebidas,15179.48,278
3,artes,24202.64,209
4,artes_e_artesanato,1814.01,24
